In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Function to load and preprocess dataset
def load_dataset(data_dir):
    images = []
    labels = []
    
    # Assuming dataset structure: data_dir/open/ and data_dir/closed/
    for state in ['open', 'closed']:
        path = os.path.join(data_dir, state)
        class_num = 0 if state == 'closed' else 1
        
        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (24, 24))  # Match the size in drowsiness detector
                img = img.astype('float32') / 255.0
                images.append(img)
                labels.append(class_num)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    
    images = np.array(images).reshape(-1, 24, 24, 1)  # Add channel dimension
    labels = np.array(labels)
    return images, labels

# Build the CNN model
def create_eye_state_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(24, 24, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model

# Main execution
if __name__ == "__main__":
    # Specify your dataset directory
    DATA_DIR = "D:\Users\Home_Desktop\Desktop\Mj\PROJECTS\Drowsiness-Detection-Essemble-Learning"  # Replace with actual path
    
    # Load and split dataset
    images, labels = load_dataset(DATA_DIR)
    X_train, X_test, y_train, y_test = train_test_split(
        images, labels, test_size=0.2, random_state=42
    )
    
    # Create and train the model
    model = create_eye_state_model()
    model.summary()
    
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=32,
        validation_data=(X_test, y_test)
    )
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_accuracy:.4f}")
    
    # Save the model
    model.save('eye_state_model.h5')
    print("Model saved as 'eye_state_model.h5'")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2218199367.py, line 55)